<a href="https://colab.research.google.com/github/YoonJiHwan98/dacon_code/blob/main/gAs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [ ]:
!git clone https://github.com/YoonJiHwan98/dacon_code.git

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

plt.rc('font', family='NanumBarunGothic') 

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/TAVE 머신러닝 스터디/energy/한국가스공사_시간별 공급량_20181231.csv',encoding = 'cp949')
data

In [ ]:
data['구분'].unique()

In [ ]:
plt.figure(figsize = (15,12))
#sns.lineplot(data = data , x = '연월일',y = '공급량',hue = '구분')
plt.show()

In [ ]:
! pip install pystan==2.19.1.1
! pip install prophet
!pip install statsmodels==0.11.1
!pip install fbprophet==0.7.1

In [ ]:
# Python
import pandas as pd
from prophet import Prophet


In [ ]:
m = Prophet(changepoint_prior_scale=0.0025, 
                        growth='logistic', 
            
                        seasonality_mode='multiplicative',
                        yearly_seasonality=True,
                        weekly_seasonality=True,  
                        daily_seasonality=False,
                        seasonality_prior_scale = 0.02,
            
                        holidays=holidays,
                        holidays_prior_scale=0.1,  
                       )
m.fit(train_1)

# 예측
future = m.make_future_dataframe(periods=31)
# future에도 동일하게 하한 제한 지정
future['cap'] = 6000
future['floor'] = 0   

forecast = m.predict(future)
forecast.tail()